In [0]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#change directory 
cd drive/My Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from scipy.spatial.distance import cdist

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional,TimeDistributed
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
url = 'senti.csv'
 
df1 = pd.read_csv(url, encoding='utf-8')     #read data from the link

print("Data size: ", len(df1))   # data size= 1,048,575

#shuffle all data
df1 = df1.reindex(np.random.permutation(df1.index.values))

Data size:  1048575


In [0]:
url = 'https://raw.githubusercontent.com/javaidnabi31/Word-Embeddding-Sentiment-Classification/master/movie_data.csv'
 
df2 = pd.read_csv(url, encoding='utf-8')     #read data from the link

print("Data size: ", len(df2))   # data size= 50,000

Data size:  50000


In [0]:
#append both kind of dataset
li = []
li.append(df1[:350000])
li.append(df2)
print(len(li))
df = pd.concat(li, axis=0, ignore_index=True)
print("dimention of data: ",df.shape)
print("Data size: ", len(df))
#print(df[0:50])

2
dimention of data:  (400000, 2)
Data size:  400000


In [0]:
#shuffle again 
df = df.sample(frac=1).reset_index(drop=True)   # or df.sample(frac=1)
#print(df[0:50])
print("Data size: ", len(df))
print("dimention of data: ",df.shape)

Data size:  400000
dimention of data:  (400000, 2)


In [0]:
x_train_text  = df.loc[:349999, 'review'].values   # intialize first 349999 data for training
y_train = df.loc[:349999, 'sentiment'].values      # intialize first 349999 data for train label

x_test_text = df.loc[350000:, 'review'].values     # intialize last 50000 data for testing
y_test = df.loc[350000:, 'sentiment'].values       # intialize last 50000 data for testing label



print("x_train_text size: ", len(x_train_text))
print("x_test_text size: ", len(x_test_text))
print("dimention of x_train_text: ",x_train_text.shape)
print("dimention of y_train: ",y_train.shape)
print("dimention of x_test_text: ",x_test_text.shape)
print("dimention of y_test: ",y_test.shape)

x_train_text size:  350000
x_test_text size:  50000
dimention of x_train_text:  (350000,)
dimention of y_train:  (350000,)
dimention of x_test_text:  (50000,)
dimention of y_test:  (50000,)


In [0]:
data_text =  df.loc[:, 'review'].values 
print("length Of Data Text: ",len(data_text) )
print("Dimention of data_text: ",data_text.shape)

length Of Data Text:  400000
Dimention of data_text:  (400000,)


In [0]:
num_words = 10000   # this is define that the first 10,000 most used words will be use
tokenizer = Tokenizer(num_words=num_words)

In [0]:
%%time
tokenizer.fit_on_texts(data_text) #making tokens of data

CPU times: user 17.7 s, sys: 59.2 ms, total: 17.7 s
Wall time: 17.7 s


In [0]:
print("length: ",len(tokenizer.word_index))
tokenizer.word_index 

length:  308440


{'the': 1,
 'a': 2,
 'to': 3,
 'and': 4,
 'i': 5,
 'of': 6,
 'is': 7,
 'in': 8,
 'it': 9,
 'br': 10,
 'this': 11,
 'that': 12,
 'for': 13,
 'was': 14,
 'but': 15,
 'you': 16,
 'with': 17,
 'on': 18,
 'my': 19,
 'as': 20,
 'have': 21,
 'movie': 22,
 'not': 23,
 'be': 24,
 'film': 25,
 'so': 26,
 'are': 27,
 'at': 28,
 'all': 29,
 'one': 30,
 'just': 31,
 'his': 32,
 'me': 33,
 'he': 34,
 'like': 35,
 'from': 36,
 'out': 37,
 'an': 38,
 'they': 39,
 'by': 40,
 "it's": 41,
 'up': 42,
 'good': 43,
 'no': 44,
 'about': 45,
 'who': 46,
 'if': 47,
 'or': 48,
 'what': 49,
 'her': 50,
 'has': 51,
 'some': 52,
 'there': 53,
 "i'm": 54,
 'time': 55,
 'when': 56,
 'get': 57,
 'more': 58,
 'really': 59,
 'very': 60,
 'had': 61,
 'do': 62,
 'see': 63,
 'can': 64,
 'now': 65,
 'too': 66,
 'only': 67,
 'would': 68,
 'well': 69,
 'we': 70,
 'even': 71,
 'she': 72,
 "don't": 73,
 'will': 74,
 'go': 75,
 'much': 76,
 'how': 77,
 'its': 78,
 'been': 79,
 'bad': 80,
 'work': 81,
 'were': 82,
 'which': 83,


In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text) # make text to sequene of tokens in train data.Detalis mention above
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)  # make text to sequene of token in test data
print(x_train_text[0])
print(np.array(x_train_tokens[0]))

@PerezHilton i know! everyone hates at him so much. and seriously the movie was pretty darn good. glad he stepped out of the HSM circle 
[6838    5   85  272 1785   28   98   26   76    4  577    1   22   14
  206 2492   43  683   34 6954   37    6    1 4519]


In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]  # counting the token numbers for every comments from 
                                                                         # both x_train_tokens & x_test_tokens
num_tokens = np.array(num_tokens)  #conver the list into array
num_tokens

array([24,  4,  9, ...,  5,  7,  2])

In [0]:
print("The average number of tokens in a sequence is: ",np.mean(num_tokens)) 
print("The maximum number of tokens in a sequence is: ",np.max(num_tokens)) 
print("The standard number of tokens in a sequence is: ",np.std(num_tokens))


The average number of tokens in a sequence is:  38.2773175
The maximum number of tokens in a sequence is:  2205
The standard number of tokens in a sequence is:  89.39297616426133


In [0]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) 
max_tokens = int(max_tokens) #TYPECAST TO INT

np.sum(num_tokens < max_tokens) / len(num_tokens) *100


217


In [0]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,padding=pad, truncating=pad)

print("dimantion of x_train_pad: ",x_train_pad.shape)  
print("dimantion of x_test_pad: ",x_test_pad.shape) 

dimantion of x_train_pad:  (350000, 217)
dimantion of x_test_pad:  (50000, 217)


In [0]:
model = Sequential()
embedding_size = 8    
model.add(Embedding(input_dim=num_words,output_dim=embedding_size,input_length=max_tokens,name='layer_embedding'))


#model.add(GRU(units=16, return_sequences=True))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Dropout(0.5))
  
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=.001)
#Compile the Keras model so it is ready for training
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 217, 8)            80000     
_________________________________________________________________
bidirectional (Bidirectional (None, 217, 128)          37376     
_________________________________________________________________
dropout (Dropout)            (None, 217, 128)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
#visulization 
from graphviz import Digraph
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Using TensorFlow backend.


In [0]:
%%time
history = model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=64)

In [0]:
history_dict = history.history
print(history_dict.keys())
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
%%time
result = model.evaluate(x_test_pad, y_test)
print("Accuracy: {0:.2%}".format(result[1]))

In [0]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]


tokens = tokenizer.texts_to_sequences(texts)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,padding=pad, truncating=pad)
tokens_pad.shape

model.predict(tokens_pad)